In [28]:
from importlib import reload

import converter
reload(converter)

<module 'converter' from '/Users/hugoschwabe/Documents/Code/AnkiAI/converter.py'>

## Generate Input Data for LLM

In [4]:
converter.generate_data()

Processing: Lehrvideos.pdf


Output()

Output()

[tesseract] lots of diacritics - possibly poor OCR

Output()

...

Output()

Output()

Output()

The output file size is 3.75× larger than the input file.
Possible reasons for this include:
--force-ocr was issued, causing transcoding.
The optional dependency 'jbig2' was not found, so some image optimizations could not be attempted.
The optional dependency 'pngquant' was not found, so some image optimizations could not be attempted.
PDF/A conversion was enabled. (Try `--output-type pdf`.)



-> Done: Lehrvideos.txt
All Files Processed!



In [17]:
data = converter.read_data()

Reading input files...
  - Read Lehrvideos.txt: 45163 characters.


## LLM Inference

#### Retrieve Information from Input Data

In [20]:
facts = converter.generate_text(converter.prompts["prompt_all"], data, "Extracting facts from input file...")

Extracting facts from input file...
Processing item 1/1: 'Lehrvideos.txt'...
  - Processing chunk 1/3...
  - Processing chunk 2/3...
  - Processing chunk 3/3...
  - Finished item 'Lehrvideos.txt'.


#### Create Flashcards from Information

In [22]:
res = converter.generate_text(converter.f_prompts["f_prompt_all"], facts, "Generating flashcards from extracted facts...")

Generating flashcards from extracted facts...
Processing item 1/1: 'Lehrvideos.txt'...
  - Processing chunk 1/1...
  - Finished item 'Lehrvideos.txt'.


## Save the Flashcards

In [29]:
converter.save_card(res)

Saving flashcards for 1 source file(s)...
    Successfully created 168 flash cards in ./output/anki_Lehrvideos.txt.
